In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hepatitis"
cohort = "GSE124719"

# Input paths
in_trait_dir = "../../input/GEO/Hepatitis"
in_cohort_dir = "../../input/GEO/Hepatitis/GSE124719"

# Output paths
out_data_file = "../../output/preprocess/Hepatitis/GSE124719.csv"
out_gene_data_file = "../../output/preprocess/Hepatitis/gene_data/GSE124719.csv"
out_clinical_data_file = "../../output/preprocess/Hepatitis/clinical_data/GSE124719.csv"
json_path = "../../output/preprocess/Hepatitis/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# Let's check what files are available in the input directory
print(f"Checking contents of {in_cohort_dir}:")
if os.path.exists(in_cohort_dir):
    dir_contents = os.listdir(in_cohort_dir)
    print(f"Files in directory: {dir_contents}")
else:
    print(f"Directory {in_cohort_dir} does not exist")

# Try to load the gene expression data if available
gene_expression_file = os.path.join(in_cohort_dir, "gene_expression.csv")
gene_series_file = os.path.join(in_cohort_dir, "series_matrix.txt")

is_gene_available = False
if os.path.exists(gene_expression_file):
    print("Gene expression file found")
    is_gene_available = True
elif os.path.exists(gene_series_file):
    print("Series matrix file found, likely contains gene expression data")
    is_gene_available = True
else:
    # Check if there are any files that might contain gene expression data
    for file in dir_contents if os.path.exists(in_cohort_dir) else []:
        if "gene" in file.lower() or "expr" in file.lower() or "matrix" in file.lower():
            print(f"Potential gene expression file found: {file}")
            is_gene_available = True
            break

# Try to load clinical data
clinical_file = os.path.join(in_cohort_dir, "clinical_data.csv")
clinical_data = None
if os.path.exists(clinical_file):
    clinical_data = pd.read_csv(clinical_file)
    print("Clinical data preview:")
    print(clinical_data.head())
else:
    print("Clinical data file not found")

# Assuming we don't have direct access to sample characteristics,
# we need to analyze clinical_data to find trait, age, and gender information

# Set default values
trait_row = None
age_row = None
gender_row = None
is_trait_available = False

# If clinical data is available, analyze it to identify trait, age, and gender rows
if clinical_data is not None:
    # Check column headers to identify potential trait, age, and gender information
    for i, column in enumerate(clinical_data.columns):
        col_lower = column.lower()
        if "hepatitis" in col_lower or "hbv" in col_lower or "hcv" in col_lower or "disease" in col_lower or "status" in col_lower:
            trait_row = i
            is_trait_available = True
        elif "age" in col_lower or "years" in col_lower:
            age_row = i
        elif "gender" in col_lower or "sex" in col_lower:
            gender_row = i
    
    # Print identified rows
    print(f"Identified rows - Trait: {trait_row}, Age: {age_row}, Gender: {gender_row}")

# Define conversion functions for clinical features
def convert_trait(value):
    """Convert trait value to binary (0 or 1)."""
    if pd.isna(value):
        return None
    
    # Convert to string to handle both string and numeric values
    value_str = str(value).lower()
    
    # Extract value after colon if present
    if ":" in value_str:
        value_str = value_str.split(":", 1)[1].strip()
    
    # Patterns for hepatitis-positive cases
    if any(pos_term in value_str for pos_term in ["positive", "hbv", "hcv", "hepatitis", "infected", "yes", "patient", "case"]):
        return 1
    # Patterns for hepatitis-negative cases
    elif any(neg_term in value_str for neg_term in ["negative", "control", "healthy", "no", "normal"]):
        return 0
    else:
        return None

def convert_age(value):
    """Convert age value to continuous numeric."""
    if pd.isna(value):
        return None
    
    # Convert to string to handle both string and numeric values
    value_str = str(value)
    
    # Extract value after colon if present
    if ":" in value_str:
        value_str = value_str.split(":", 1)[1].strip()
    
    # Try to extract numeric age
    import re
    age_match = re.search(r'(\d+\.?\d*)', value_str)
    if age_match:
        return float(age_match.group(1))
    else:
        return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)."""
    if pd.isna(value):
        return None
    
    # Convert to string to handle both string and numeric values
    value_str = str(value).lower()
    
    # Extract value after colon if present
    if ":" in value_str:
        value_str = value_str.split(":", 1)[1].strip()
    
    if any(male_term in value_str for male_term in ["male", "m", "man", "men"]):
        return 1
    elif any(female_term in value_str for female_term in ["female", "f", "woman", "women"]):
        return 0
    else:
        return None

# 3. Save Metadata
# Validate and save cohort info (initial filtering)
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if is_trait_available and clinical_data is not None:
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(clinical_features)
    print("\nExtracted Clinical Features Preview:")
    print(preview)
    
    # Save clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print("Trait data not available or clinical data not found. Skipping clinical feature extraction.")


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


### Step 5: Gene Identifier Review

In [ ]:
# Based on the gene IDs shown in the output from the previous step, 
# they appear to be numeric identifiers (1, 2, 3, etc.) rather than human gene symbols.
# These are likely probe or feature IDs from a microarray platform that need to be 
# mapped to actual gene symbols for biological interpretation.

# In the GEO database, these numeric IDs typically correspond to probes on a microarray,
# and they need to be mapped to gene symbols using platform-specific annotation information.

requires_gene_mapping = True
print(f"Based on gene identifier review, mapping requirement: {requires_gene_mapping}")


### Step 6: Gene Annotation

In [ ]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # First attempt - use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Look for columns that might contain gene symbols
    gene_symbol_columns = [col for col in gene_annotation.columns if 
                          any(term in col.lower() for term in ['symbol', 'gene', 'genename', 'gene_symbol'])]
    
    if gene_symbol_columns:
        print(f"\nPotential gene symbol columns: {gene_symbol_columns}")
        # Show examples from these columns
        for col in gene_symbol_columns:
            print(f"\nSample values from {col} column:")
            print(gene_annotation[col].dropna().head(10).tolist())
    else:
        print("\nNo obvious gene symbol columns found. Examining all columns for gene symbol patterns...")
        # Check a few rows of all columns for potential gene symbols
        for col in gene_annotation.columns:
            sample_values = gene_annotation[col].dropna().head(5).astype(str).tolist()
            print(f"\nSample values from {col} column: {sample_values}")
            
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    
    # Alternative approach if the library function fails
    print("\nTrying alternative approach to find gene annotation...")
    with gzip.open(soft_file, 'rt') as file:
        # Look for platform ID
        platform_id = None
        for line in file:
            if line.startswith('!Series_platform_id'):
                platform_id = line.split('=')[1].strip()
                print(f"Platform ID: {platform_id}")
                break
        
        # If we found a platform ID, look for that section
        if platform_id:
            file.seek(0)  # Go back to start of file
            in_platform_section = False
            for line in file:
                if line.startswith(f'^PLATFORM = {platform_id}'):
                    in_platform_section = True
                    print(f"Found platform section: {line.strip()}")
                    break
            
            # If we found the platform section, print some annotation info
            if in_platform_section:
                for i, line in enumerate(file):
                    if i < 20 and (line.startswith('!Platform_title') or 
                                  line.startswith('!Platform_organism') or
                                  line.startswith('!Platform_technology') or
                                  'annotation' in line.lower()):
                        print(line.strip())


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Identify the columns in gene annotation for mapping
id_column = 'ID'  # The numeric identifiers in gene expression data
gene_symbol_column = 'GENE_SYMBOL'  # The standard gene symbols

print("Mapping probe IDs to gene symbols...")
print(f"Using {id_column} as probe identifier and {gene_symbol_column} as gene symbol")

# 2. Get the gene mapping dataframe 
mapping_df = get_gene_mapping(gene_annotation, id_column, gene_symbol_column)
print(f"Created mapping with {len(mapping_df)} entries")
print("First few mapping entries:")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Successfully created gene expression data with {len(gene_data.index)} genes")
print("First few genes in the expression data:")
print(gene_data.index[:10].tolist())

# Check shape of the final gene expression data
print(f"Gene expression data shape: {gene_data.shape}")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols and extract from step 3 and 6
# Load the gene expression data (already loaded from Step 6)
print(f"Gene data shape before normalization: {gene_data.shape}")

try:
    # Normalize gene symbols using the NCBI Gene database information
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    
    if normalized_gene_data.empty:
        print("Normalization resulted in empty dataframe. Using original gene data instead.")
        normalized_gene_data = gene_data
    
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data to the output file
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene data: {e}. Using original gene data instead.")
    normalized_gene_data = gene_data
    # Save the original gene data if normalization fails
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)

# 2. Link clinical and genetic data
# Use the trait_row identified in Step 2 (trait_row = 1) to extract trait data
is_trait_available = trait_row is not None

if is_trait_available:
    # Extract clinical features using the function and conversion methods from Step 2
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
else:
    # Create a minimal dataframe with just the trait column
    linked_data = pd.DataFrame({trait: [np.nan]})
    print("No trait data available, creating minimal dataframe for validation.")

# 3. Handle missing values in the linked data
if is_trait_available:
    print("\nHandling missing values...")
    linked_data = handle_missing_values(linked_data, trait)
    print(f"After missing value handling, linked data shape: {linked_data.shape}")

# 4. Determine whether trait and demographic features are biased
if is_trait_available and not linked_data.empty and len(linked_data.columns) > 1:
    print("\nEvaluating feature bias...")
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Trait bias evaluation result: {is_biased}")
else:
    is_biased = False
    print("Skipping bias evaluation due to insufficient data.")

# 5. Final validation and save metadata
note = ""
if not is_trait_available:
    note = f"Dataset contains gene expression data but no {trait} measurements."
elif is_biased:
    note = f"Dataset contains {trait} data but its distribution is severely biased."

# Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save the linked data if usable
print(f"\nDataset usability: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset is not usable for {trait} association studies. Data not saved.")